In [1]:
# ATSIT 2019: dimensional speech emotion recognition from text feature

# uncomment these to run on CPU only
import os
#os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
#os.environ["CUDA_VISIBLE_DEVICES"]=""

import numpy as np
import matplotlib.pyplot as plt
import pickle
import keras.backend as K
import pandas as pd
from keras.models import Model, Sequential
from keras.layers import Input, Dense, Masking, LSTM, TimeDistributed, Bidirectional, Embedding, Dropout, Flatten, concatenate, Convolution1D
from keras.utils import to_categorical
from sklearn.preprocessing import MinMaxScaler

from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence

from keras.callbacks import EarlyStopping
import codecs

import io
import random as rn
import tensorflow as tf

Using TensorFlow backend.


In [2]:
rn.seed(123)
np.random.seed(99)
tf.set_random_seed(1234)

In [3]:

# Concordance correlation coefficient (CCC)-based loss function - using non-inductive statistics
def ccc(gold, pred):
    gold       = K.squeeze(gold, axis=-1)
    pred       = K.squeeze(pred, axis=-1)
    gold_mean  = K.mean(gold, axis=-1, keepdims=True)
    pred_mean  = K.mean(pred, axis=-1, keepdims=True)
    covariance = (gold-gold_mean)*(pred-pred_mean)
    gold_var   = K.mean(K.square(gold-gold_mean), axis=-1,  keepdims=True)
    pred_var   = K.mean(K.square(pred-pred_mean), axis=-1, keepdims=True)
    ccc        = K.constant(2.) * covariance / (gold_var + pred_var + K.square(gold_mean - pred_mean) + K.common.epsilon())
    return ccc


def ccc_loss(gold, pred):  
    # input (num_batches, seq_len, 1)
    ccc_loss   = K.constant(1.) - ccc(gold, pred)
    return ccc_loss

#path = '/home/s1820002/IEMOCAP-Emotion-Detection/'
data = np.load('../../dimensionalSM/IEMOCAP_full_release/data_collected_10039.pickle', allow_pickle=True)
text = [t['transcription'] for t in data]
vad = np.load('../../dimensionalSM/data/y_egemaps.npy')

# remove outlier, < 1, > 5
vad = np.where(vad==5.5, 5.0, vad)
vad = np.where(vad==0.5, 1.0, vad)

scaled_vad = True

# standardization
if scaled_vad:
    scaler = MinMaxScaler(feature_range=(-1, 1))
    scaler = scaler.fit(vad) #.reshape(vad.shape[0]*vad.shape[1], vad.shape[2]))
    scaled_vad = scaler.transform(vad) #.reshape(vad.shape[0]*vad.shape[1], vad.shape[2]))
    vad = scaled_vad 
else:
    vad = vad

# load fastext model
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text)

token_tr_X = tokenizer.texts_to_sequences(text)
MAX_SEQUENCE_LENGTH = len(max(token_tr_X, key=len))
x_train_text = []
x_train_text = sequence.pad_sequences(token_tr_X, maxlen=MAX_SEQUENCE_LENGTH)

word_index = tokenizer.word_index
nb_words = len(word_index) +1
print('Found %s unique tokens' % len(word_index))

# load model
EMBEDDING_DIM = 300
file_loc = '../../dimensionalSM/data/glove.42B.300d.txt'
print (file_loc)

gembeddings_index = {}
with codecs.open(file_loc, encoding='utf-8') as f:
    for line in f:
        values = line.split(' ')
        word = values[0]
        gembedding = np.asarray(values[1:], dtype='float32')
        gembeddings_index[word] = gembedding
#
f.close()
print('G Word embeddings:', len(gembeddings_index))

nb_words = len(word_index) +1
g_word_embedding_matrix = np.zeros((nb_words, EMBEDDING_DIM))
for word, i in word_index.items():
    gembedding_vector = gembeddings_index.get(word)
    if gembedding_vector is not None:
        g_word_embedding_matrix[i] = gembedding_vector
        
print('G Null word embeddings: %d' % np.sum(np.sum(g_word_embedding_matrix, axis=1) == 0))

# split train/test
split = 7869
earlystop = EarlyStopping(monitor='val_loss', patience=10, mode='min', 
                          restore_best_weights=True)



Found 3437 unique tokens
../../dimensionalSM/data/glove.42B.300d.txt
G Word embeddings: 1917494
G Null word embeddings: 128


In [4]:
MAX_SEQUENCE_LENGTH

100

In [4]:
# model: GRU
def text_model1():
    inputs = Input(shape=(MAX_SEQUENCE_LENGTH, ))
    net = Embedding(nb_words,
                    EMBEDDING_DIM,
                    weights = [g_word_embedding_matrix],
                    trainable = True)(inputs)
    #net = Bidirectional(LSTM(64, return_sequences=True))(net)
#     net = Bidirectional(LSTM(64, return_sequences=True))(net)
#     net = Bidirectional(LSTM(64, return_sequences=True))(net)
    net = Bidirectional(LSTM(128, return_sequences=True))(net)
    net = Bidirectional(LSTM(64, return_sequences=False))(net)  
    out_text = Dense(64)(net)
    #out_text = Dropout(0.5)(net)
    target_names = ('v', 'a', 'd')
    outputs = [Dense(1, name=name)(out_text) for name in target_names]

    model = Model(inputs=inputs, outputs=outputs) #=[out1, out2, out3])
    model.compile(loss=ccc_loss, #{'v': ccc_loss, 'a': ccc_loss, 'd': ccc_loss},
                     loss_weights={'v': 1, 'a': 1, 'd': 1},
                  optimizer='rmsprop', metrics=[ccc])
    return model
model = text_model1()
model.summary()

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 100)          0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 100, 300)     1031400     input_1[0][0]                    
__________________________________________________________________________________________________
bidirectional_1 (Bidirectional) (None, 100, 256)     439296      embedding_1[0][0]                
__________________________________________________________________________________________________
bidirectional_2 (Bidirectional) (None, 128)          164352      bidirectional_1[0][0]            
___________

In [5]:
# #speech_to_npy
# model1 = text_model1()
# earlystop = EarlyStopping(monitor='val_loss', mode='min', patience=10,
#                               restore_best_weights=True)
# hist1 = model1.fit(x_train_text[:8000], vad[:8000].T.tolist(), epochs=50, 
#                    batch_size=32, verbose=1, validation_split=0.2, 
#                    callbacks=[earlystop])

# eval_metrik1 = model1.evaluate(x_train_text[8000:], vad[8000:].T.tolist())
# print(eval_metrik1)

# # make prediction
# predict = model1.predict(x_train_text[6400:], batch_size=32)
# np.save('text_npy', 
#          np.array(predict).reshape(3, 3639).T)


In [6]:
# main function for for LOSO
def main(time):
    model = text_model1()
    earlystop = EarlyStopping(monitor='val_loss', mode='min', patience=10,
                              restore_best_weights=True)
    hist = model.fit( x_train_text[:8000], 
                      vad[:8000].T.tolist(), batch_size=32, #best:8
                      validation_split=0.2, epochs=50, verbose=1, shuffle=True,
                      callbacks=[earlystop])
    metrik = model.evaluate( x_train_text[8000:], vad[8000:].T.tolist())
    print(metrik[-3:])
    # make prediction
    predict = model.predict(x_train_text[6400:], batch_size=32)
    np.save('npy8000/text_npy-glove-8000'+str(time)+'.npy',  
             np.array(predict).reshape(3, 3639).T)
    return metrik
    
if __name__ == '__main__':
    #loss_weight_step = np.around(np.arange(0.0, 1.1, 0.1), decimals=1)
    weight = []
    for time in range(20):
        best = main(time)
        weight.append([time, best[-3], best[-2], best[-1], np.mean(best[-3:])])
        print(time)
    weight_df = pd.DataFrame(weight)
    weight_df.to_csv('npy8000/text_blstm8000_no_dropout-1.csv', header=['time','v','a','d', 'ave'])


Instructions for updating:
If using Keras pass *_constraint arguments to layers.

Train on 6400 samples, validate on 1600 samples
Epoch 1/50
6400/6400 [==============================] - 54s 8ms/step - loss: 1.8903 - v_loss: 0.6019 - a_loss: 0.6447 - d_loss: 0.6437 - v_ccc: 0.3981 - a_ccc: 0.3553 - d_ccc: 0.3563 - val_loss: 1.9845 - val_v_loss: 0.6788 - val_a_loss: 0.6522 - val_d_loss: 0.6534 - val_v_ccc: 0.3212 - val_a_ccc: 0.3478 - val_d_ccc: 0.3466
Epoch 2/50
6400/6400 [==============================] - 51s 8ms/step - loss: 1.4564 - v_loss: 0.4129 - a_loss: 0.5240 - d_loss: 0.5195 - v_ccc: 0.5871 - a_ccc: 0.4760 - d_ccc: 0.4805 - val_loss: 1.9333 - val_v_loss: 0.6428 - val_a_loss: 0.6331 - val_d_loss: 0.6573 - val_v_ccc: 0.3572 - val_a_ccc: 0.3669 - val_d_ccc: 0.3427
Epoch 3/50
6400/6400 [==============================] - 50s 8ms/step - loss: 1.2587 - v_loss: 0.3316 - a_loss: 0.4712 - d_loss: 0.4558 - v_ccc: 0.6684 - a_ccc: 0.5288 - d_ccc: 0.5442 - val_loss: 1.9913 - val_v_loss: 0.67

Epoch 5/50
6400/6400 [==============================] - 51s 8ms/step - loss: 1.0370 - v_loss: 0.2577 - a_loss: 0.3963 - d_loss: 0.3830 - v_ccc: 0.7423 - a_ccc: 0.6037 - d_ccc: 0.6170 - val_loss: 1.8871 - val_v_loss: 0.5920 - val_a_loss: 0.6231 - val_d_loss: 0.6720 - val_v_ccc: 0.4080 - val_a_ccc: 0.3769 - val_d_ccc: 0.3280
Epoch 6/50
6400/6400 [==============================] - 51s 8ms/step - loss: 0.9420 - v_loss: 0.2360 - a_loss: 0.3532 - d_loss: 0.3527 - v_ccc: 0.7640 - a_ccc: 0.6468 - d_ccc: 0.6473 - val_loss: 1.9097 - val_v_loss: 0.5795 - val_a_loss: 0.6569 - val_d_loss: 0.6733 - val_v_ccc: 0.4205 - val_a_ccc: 0.3431 - val_d_ccc: 0.3267
Epoch 7/50
6400/6400 [==============================] - 51s 8ms/step - loss: 0.8593 - v_loss: 0.2127 - a_loss: 0.3256 - d_loss: 0.3211 - v_ccc: 0.7873 - a_ccc: 0.6744 - d_ccc: 0.6789 - val_loss: 1.9111 - val_v_loss: 0.5837 - val_a_loss: 0.6395 - val_d_loss: 0.6879 - val_v_ccc: 0.4163 - val_a_ccc: 0.3605 - val_d_ccc: 0.3121
Epoch 8/50
6400/6400 [===

Epoch 17/50
2039/2039 [==============================] - 4s 2ms/step
[0.419017493724823, 0.3613010048866272, 0.40198206901550293]
2
Train on 6400 samples, validate on 1600 samples
Epoch 1/50
6400/6400 [==============================] - 55s 9ms/step - loss: 1.9056 - v_loss: 0.6156 - a_loss: 0.6438 - d_loss: 0.6462 - v_ccc: 0.3844 - a_ccc: 0.3562 - d_ccc: 0.3538 - val_loss: 2.0410 - val_v_loss: 0.7208 - val_a_loss: 0.6556 - val_d_loss: 0.6646 - val_v_ccc: 0.2792 - val_a_ccc: 0.3444 - val_d_ccc: 0.3354
Epoch 2/50
6400/6400 [==============================] - 50s 8ms/step - loss: 1.4489 - v_loss: 0.4082 - a_loss: 0.5229 - d_loss: 0.5178 - v_ccc: 0.5918 - a_ccc: 0.4771 - d_ccc: 0.4822 - val_loss: 1.9627 - val_v_loss: 0.6577 - val_a_loss: 0.6583 - val_d_loss: 0.6467 - val_v_ccc: 0.3423 - val_a_ccc: 0.3417 - val_d_ccc: 0.3533
Epoch 3/50
6400/6400 [==============================] - 50s 8ms/step - loss: 1.2653 - v_loss: 0.3353 - a_loss: 0.4701 - d_loss: 0.4599 - v_ccc: 0.6647 - a_ccc: 0.5299 - d

Epoch 12/50
6400/6400 [==============================] - 50s 8ms/step - loss: 0.5521 - v_loss: 0.1331 - a_loss: 0.2098 - d_loss: 0.2093 - v_ccc: 0.8669 - a_ccc: 0.7902 - d_ccc: 0.7907 - val_loss: 1.9170 - val_v_loss: 0.5859 - val_a_loss: 0.6466 - val_d_loss: 0.6845 - val_v_ccc: 0.4141 - val_a_ccc: 0.3534 - val_d_ccc: 0.3155
Epoch 13/50
6400/6400 [==============================] - 51s 8ms/step - loss: 0.5062 - v_loss: 0.1240 - a_loss: 0.1928 - d_loss: 0.1895 - v_ccc: 0.8760 - a_ccc: 0.8072 - d_ccc: 0.8105 - val_loss: 1.9221 - val_v_loss: 0.5654 - val_a_loss: 0.6690 - val_d_loss: 0.6877 - val_v_ccc: 0.4346 - val_a_ccc: 0.3310 - val_d_ccc: 0.3123
Epoch 14/50
6400/6400 [==============================] - 50s 8ms/step - loss: 0.4754 - v_loss: 0.1172 - a_loss: 0.1832 - d_loss: 0.1750 - v_ccc: 0.8828 - a_ccc: 0.8168 - d_ccc: 0.8250 - val_loss: 1.9319 - val_v_loss: 0.5733 - val_a_loss: 0.6628 - val_d_loss: 0.6959 - val_v_ccc: 0.4267 - val_a_ccc: 0.3372 - val_d_ccc: 0.3041
Epoch 15/50
6400/6400 

Epoch 7/50
6400/6400 [==============================] - 40s 6ms/step - loss: 0.8466 - v_loss: 0.2063 - a_loss: 0.3204 - d_loss: 0.3198 - v_ccc: 0.7937 - a_ccc: 0.6796 - d_ccc: 0.6802 - val_loss: 1.9128 - val_v_loss: 0.6108 - val_a_loss: 0.6207 - val_d_loss: 0.6813 - val_v_ccc: 0.3892 - val_a_ccc: 0.3793 - val_d_ccc: 0.3187
Epoch 8/50
6400/6400 [==============================] - 40s 6ms/step - loss: 0.7764 - v_loss: 0.1881 - a_loss: 0.2935 - d_loss: 0.2948 - v_ccc: 0.8119 - a_ccc: 0.7065 - d_ccc: 0.7052 - val_loss: 1.8889 - val_v_loss: 0.5862 - val_a_loss: 0.6256 - val_d_loss: 0.6771 - val_v_ccc: 0.4138 - val_a_ccc: 0.3744 - val_d_ccc: 0.3229
Epoch 9/50
6400/6400 [==============================] - 40s 6ms/step - loss: 0.7045 - v_loss: 0.1749 - a_loss: 0.2655 - d_loss: 0.2641 - v_ccc: 0.8251 - a_ccc: 0.7345 - d_ccc: 0.7359 - val_loss: 1.8735 - val_v_loss: 0.5667 - val_a_loss: 0.6332 - val_d_loss: 0.6736 - val_v_ccc: 0.4333 - val_a_ccc: 0.3668 - val_d_ccc: 0.3264
Epoch 10/50
6400/6400 [==

Epoch 16/50
6400/6400 [==============================] - 45s 7ms/step - loss: 0.4197 - v_loss: 0.1018 - a_loss: 0.1625 - d_loss: 0.1554 - v_ccc: 0.8982 - a_ccc: 0.8375 - d_ccc: 0.8446 - val_loss: 1.9724 - val_v_loss: 0.5818 - val_a_loss: 0.6934 - val_d_loss: 0.6973 - val_v_ccc: 0.4182 - val_a_ccc: 0.3066 - val_d_ccc: 0.3027
Epoch 17/50
2039/2039 [==============================] - 4s 2ms/step
[0.4256524443626404, 0.36946260929107666, 0.399128258228302]
9
Train on 6400 samples, validate on 1600 samples
Epoch 1/50
6400/6400 [==============================] - 49s 8ms/step - loss: 1.8858 - v_loss: 0.5987 - a_loss: 0.6465 - d_loss: 0.6406 - v_ccc: 0.4013 - a_ccc: 0.3535 - d_ccc: 0.3594 - val_loss: 1.9595 - val_v_loss: 0.6799 - val_a_loss: 0.6376 - val_d_loss: 0.6420 - val_v_ccc: 0.3201 - val_a_ccc: 0.3624 - val_d_ccc: 0.3580
Epoch 2/50
6400/6400 [==============================] - 44s 7ms/step - loss: 1.4431 - v_loss: 0.4024 - a_loss: 0.5247 - d_loss: 0.5160 - v_ccc: 0.5976 - a_ccc: 0.4753 - 

Epoch 15/50
2039/2039 [==============================] - 4s 2ms/step
[0.4055275619029999, 0.39292222261428833, 0.39495939016342163]
12
Train on 6400 samples, validate on 1600 samples
Epoch 1/50
6400/6400 [==============================] - 51s 8ms/step - loss: 1.8870 - v_loss: 0.6159 - a_loss: 0.6345 - d_loss: 0.6366 - v_ccc: 0.3841 - a_ccc: 0.3655 - d_ccc: 0.3634 - val_loss: 2.0068 - val_v_loss: 0.6846 - val_a_loss: 0.6554 - val_d_loss: 0.6668 - val_v_ccc: 0.3154 - val_a_ccc: 0.3446 - val_d_ccc: 0.3332
Epoch 2/50
6400/6400 [==============================] - 43s 7ms/step - loss: 1.4615 - v_loss: 0.4148 - a_loss: 0.5267 - d_loss: 0.5199 - v_ccc: 0.5852 - a_ccc: 0.4733 - d_ccc: 0.4801 - val_loss: 2.0115 - val_v_loss: 0.6259 - val_a_loss: 0.6915 - val_d_loss: 0.6941 - val_v_ccc: 0.3741 - val_a_ccc: 0.3085 - val_d_ccc: 0.3059
Epoch 3/50
6400/6400 [==============================] - 42s 7ms/step - loss: 1.2603 - v_loss: 0.3381 - a_loss: 0.4643 - d_loss: 0.4578 - v_ccc: 0.6619 - a_ccc: 0.5357 

2039/2039 [==============================] - 4s 2ms/step
[0.4196390211582184, 0.39035293459892273, 0.37624287605285645]
15
Train on 6400 samples, validate on 1600 samples
Epoch 1/50
6400/6400 [==============================] - 49s 8ms/step - loss: 1.8835 - v_loss: 0.6015 - a_loss: 0.6460 - d_loss: 0.6359 - v_ccc: 0.3985 - a_ccc: 0.3540 - d_ccc: 0.3641 - val_loss: 1.9949 - val_v_loss: 0.6823 - val_a_loss: 0.6528 - val_d_loss: 0.6599 - val_v_ccc: 0.3177 - val_a_ccc: 0.3472 - val_d_ccc: 0.3401
Epoch 2/50
6400/6400 [==============================] - 44s 7ms/step - loss: 1.4309 - v_loss: 0.3988 - a_loss: 0.5180 - d_loss: 0.5141 - v_ccc: 0.6012 - a_ccc: 0.4820 - d_ccc: 0.4859 - val_loss: 1.9229 - val_v_loss: 0.6293 - val_a_loss: 0.6309 - val_d_loss: 0.6626 - val_v_ccc: 0.3707 - val_a_ccc: 0.3691 - val_d_ccc: 0.3374
Epoch 3/50
6400/6400 [==============================] - 46s 7ms/step - loss: 1.2684 - v_loss: 0.3336 - a_loss: 0.4765 - d_loss: 0.4583 - v_ccc: 0.6664 - a_ccc: 0.5235 - d_ccc: 0.5

6400/6400 [==============================] - 45s 7ms/step - loss: 1.2750 - v_loss: 0.3437 - a_loss: 0.4736 - d_loss: 0.4577 - v_ccc: 0.6563 - a_ccc: 0.5264 - d_ccc: 0.5423 - val_loss: 1.8447 - val_v_loss: 0.5958 - val_a_loss: 0.6050 - val_d_loss: 0.6438 - val_v_ccc: 0.4042 - val_a_ccc: 0.3950 - val_d_ccc: 0.3562
Epoch 4/50
6400/6400 [==============================] - 45s 7ms/step - loss: 1.1456 - v_loss: 0.2887 - a_loss: 0.4312 - d_loss: 0.4257 - v_ccc: 0.7113 - a_ccc: 0.5688 - d_ccc: 0.5743 - val_loss: 1.8685 - val_v_loss: 0.5812 - val_a_loss: 0.6228 - val_d_loss: 0.6645 - val_v_ccc: 0.4188 - val_a_ccc: 0.3772 - val_d_ccc: 0.3355
Epoch 5/50
6400/6400 [==============================] - 45s 7ms/step - loss: 1.0284 - v_loss: 0.2570 - a_loss: 0.3867 - d_loss: 0.3847 - v_ccc: 0.7430 - a_ccc: 0.6133 - d_ccc: 0.6153 - val_loss: 1.8578 - val_v_loss: 0.5735 - val_a_loss: 0.6224 - val_d_loss: 0.6619 - val_v_ccc: 0.4265 - val_a_ccc: 0.3776 - val_d_ccc: 0.3381
Epoch 6/50
6400/6400 [==============